# 세이지메이커 훈련 작업을 사용하여 대화 요약을 위한 인스트럭션 기반 학습 모델 미세 조정

# 주의: 이 노트북을 완료하는 데 약 20분이 소요됩니다.

# 잠시 기다려 주세요.

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='gaia/1.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region, 
                            config=config)

# 사전 요구 사항: 이 노트북을 진행하기 전에 `준비` 섹션의 노트북을 성공적으로 실행해야 합니다.

In [ ]:
%store -r processed_train_data_s3_uri

In [ ]:
try:
    processed_train_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] 계속 진행하기 전에 '준비' 섹션의 노트북을 실행해 주세요.                     ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(processed_train_data_s3_uri)

In [ ]:
%store -r processed_validation_data_s3_uri

In [ ]:
try:
    processed_validation_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] 계속 진행하기 전에 '준비' 섹션의 노트북을 실행해 주세요.                     ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(processed_validation_data_s3_uri)

In [ ]:
%store -r processed_test_data_s3_uri

In [ ]:
try:
    processed_test_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] 계속 진행하기 전에 '준비' 섹션의 노트북을 실행해 주세요.                     ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(processed_test_data_s3_uri)

# S3에 있는 데이터 세트 지정하기
이전 노트북에서 생성한 훈련, 검증, 테스트 분할을 사용합니다.

In [ ]:
print(processed_train_data_s3_uri)

!aws s3 ls $processed_train_data_s3_uri/

In [ ]:
print(processed_validation_data_s3_uri)

!aws s3 ls $processed_validation_data_s3_uri/

In [ ]:
print(processed_test_data_s3_uri)

!aws s3 ls $processed_test_data_s3_uri/

# S3 입력 데이터 지정하기

In [ ]:
from sagemaker.inputs import TrainingInput

s3_input_train_data = TrainingInput(s3_data=processed_train_data_s3_uri)
s3_input_validation_data = TrainingInput(s3_data=processed_validation_data_s3_uri)
s3_input_test_data = TrainingInput(s3_data=processed_test_data_s3_uri)

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

# FLAN 모델을 위한 하이퍼파라미터 설정하기

In [ ]:
model_checkpoint='google/flan-t5-base'

In [ ]:
epochs = 1 # 더 오랜 기간 동안 훈련하고 싶다면 이 값을 늘리세요.
learning_rate = 0.00001
weight_decay = 0.01
train_batch_size = 4
validation_batch_size = 4
test_batch_size = 4
train_instance_count = 1
train_instance_type = "ml.c5.9xlarge"
train_volume_size = 1024
input_mode = "FastFile"
train_sample_percentage = 0.01 # 더 많은 데이터로 훈련하고 싶다면 이 값을 늘리세요.

# 모델 성능 추적을 위한 메트릭 설정하기

In [ ]:
metrics_definitions = [
    {"Name": "train:loss", "Regex": "'train_loss': ([0-9\\.]+)"},
    {"Name": "validation:loss", "Regex": "'eval_loss': ([0-9\\.]+)"},
]

# 체크포인트 S3 위치 지정하기
이 노트북에는 스팟 인스턴스 훈련을 사용합니다. 노드가 교체되면, 새 노드는 최신 체크포인트에서 훈련을 시작합니다.

In [ ]:
import uuid

checkpoint_s3_prefix = "checkpoints/{}".format(str(uuid.uuid4()))
checkpoint_s3_uri = "s3://{}/{}/".format(bucket, checkpoint_s3_prefix)

print(checkpoint_s3_uri)

# 세이지메이커에서 실행할 스크립트 설정하기
관리형 세이지메이커 서비스에서 모델을 실행할 준비합니다.

In [ ]:
!pygmentize src/train.py

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="train.py",
    source_dir="src",
    role=role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    volume_size=train_volume_size,
    checkpoint_s3_uri=checkpoint_s3_uri,
    py_version="py39",
    framework_version="1.13",
    hyperparameters={
        "epochs": epochs,
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,        
        "train_batch_size": train_batch_size,
        "validation_batch_size": validation_batch_size,
        "test_batch_size": test_batch_size,
        "model_checkpoint": model_checkpoint,
        "train_sample_percentage": train_sample_percentage,
    },
    input_mode=input_mode,
    metric_definitions=metrics_definitions,
)

# 세이지메이커에서 모델 훈련하기

In [ ]:
estimator.fit(
    inputs={"train": s3_input_train_data, "validation": s3_input_validation_data, "test": s3_input_test_data},
    wait=False,
)

In [ ]:
training_job_name = estimator.latest_training_job.name
print("Training Job Name: {}".format(training_job_name))

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>약 5분 후 <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">훈련 작업</a> 검토하기</b>'.format(
            region, training_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>약 5분 후 <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch 로그</a> 검토하기</b>'.format(
            region, training_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>훈련 작업이 완료된 후 <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 출력 데이터</a> 검토하기</b>'.format(
            bucket, training_job_name, region
        )
    )
)

In [ ]:
%%time

estimator.latest_training_job.wait(logs=False)

# 미세 조정된 모델을 실시간 엔드포인트에 배포하기

In [ ]:
sm_model = estimator.create_model(
    entry_point='inference.py',
    source_dir='src',
)
endpoint_name = training_job_name.replace('pytorch-training', 'summary-tuned')
predictor = sm_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.2xlarge',
    endpoint_name=endpoint_name
)

# 세이지메이커 엔드포인트에서 미세 조정된 모델로 제로샷 추론하기

In [ ]:
zero_shot_prompt = """Summarize the following conversation.

#Person1#: Tom, I've got good news for you.
#Person2#: What is it?
#Person1#: Haven't you heard that your novel has won The Nobel Prize?
#Person2#: Really? I can't believe it. It's like a dream come true. I never expected that I would win The Nobel Prize!
#Person1#: You did a good job. I'm extremely proud of you.
#Person2#: Thanks for the compliment.
#Person1#: You certainly deserve it. Let's celebrate!

Summary:"""

In [ ]:
import json
from sagemaker import Predictor
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
)
response = predictor.predict(zero_shot_prompt,
        {
            "ContentType": "application/x-text",
            "Accept": "application/json",
        },
)
response_json = json.loads(response.decode('utf-8'))
print(response_json)

# 엔드포인트 종료하기

In [ ]:
# predictor.delete_endpoint()